## RAG

## Author Name:

#### Tianzheng Hou

## Setting up the libraries and the environment

In [45]:
import os
import pandas as pd
import numpy as np
import tiktoken
from uuid import uuid4
# from tqdm import tqdm
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm
import string
import re
import faiss

# LANGCHAIN
import langchain
from langchain.llms import OpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import FAISS

# VECTOR STORE
import pinecone
from pinecone import Pinecone, ServerlessSpec

# HUGGINGFACE
from transformers import AutoTokenizer, AutoModel
# from langchain_transformers import HuggingFaceEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# AGENTS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, Tool, AgentType
from langchain.agents.react.agent import create_react_agent
from langchain import hub

## Data Preprocessing and Model Selection

In [46]:
loader = CSVLoader(
    file_path="Food Ingredients and Recipe Dataset with Image Name Mapping.csv",
    encoding='utf-8',
    source_column="Instructions",
    metadata_columns=["Title", "Ingredients", "Cleaned_Ingredients"]
)


data = loader.load()

In [47]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,
    chunk_overlap=20,
    separators= ["\n\n", "\n", " ", ""])

index_name = "langchain-pinecone-test"
os.environ['PINECONE_API_KEY'] = "72faa6b9-16d9-4293-a0bc-bbcd27a7589f"
pc = Pinecone()

In [ ]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"))

In [48]:
pc.list_indexes()
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'first_split': {'vector_count': 124},
                'last_split': {'vector_count': 116},
                'main': {'vector_count': 600},
                'main_alt': {'vector_count': 304}},
 'total_vector_count': 1144}

In [49]:
splits = text_splitter.split_documents(data[:100])

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed = HuggingFaceEmbeddings(model_name=model_name)

db = PineconeVectorStore.from_documents(documents=splits,
                                        embedding=embed,
                                        index_name=index_name,
                                        namespace="main"
                                        )
vectorstore = PineconeVectorStore(index_name=index_name,
                                  namespace="main",
                                  embedding=embed)

C:\Users\hou18\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [50]:
def chunked_metadata_embeddings(documents, embed):

    chunked_metadata = []

    chunked_text = text_splitter.split_documents(documents)

    for index, text in enumerate(tqdm(chunked_text)):


        payload = {
              "metadata": {
                  "source": text.metadata['source'],
                  "row": text.metadata['row'],
                  "chunk_num": index,
                  "Title": text.metadata['Title'],
                  "Ingredients": text.metadata['Ingredients'],
                  "Cleaned_Ingredients": text.metadata['Cleaned_Ingredients'],
              },
              "id": str(uuid4()),
              "values": embed.embed_documents([text.page_content])[0]  # Assuming `embed` is defined elsewhere
          }

        chunked_metadata.append(payload)

    return chunked_metadata

In [51]:
half = int(len(data)/2 - 0.5)
half = 50
split_one = chunked_metadata_embeddings(data[:half], embed)
len(split_one)
split_two = chunked_metadata_embeddings(data[half:100], embed)
len(split_two)

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

58

In [52]:
def batch_upsert(split,
                 index ,
                 namespace,
                 batch_size):

    print(f"Split Length: {len(split)}")
    for i in range(0, len(split), batch_size):

      batch = split[i:i + batch_size]

      index.upsert(vectors=batch,
                   namespace=namespace)

In [53]:
batch_upsert(split_one, index, "first_split", 10)

Split Length: 62


In [54]:
batch_upsert(split_two, index, "last_split", 20)

Split Length: 58


## Implementing RAG using LangChain for different queries

Retrieval Augmented Generation (RAG) is a technique that enhances the capabilities of Large Language Models (LLMs) by combining them with a retrieval system. This approach enables LLMs to access and process relevant information from external data sources, resulting in more accurate, informative, and up-to-date responses. <br>
1.Data Preprocessing: clean, structure, and enrichment the data. <br>
2.Embedding: Converting text data into numerical representations (embeddings) that capture semantic and syntactic information. Storing these embeddings in a vector database for efficient search and retrieval. <br>
3.Retrieval: Converting the user's query into an embedding. Finding the most similar embeddings in the vector database, corresponding to relevant documents or information. Ordering the retrieved documents based on their relevance to the query.<br>
4.Language Model (LLM): Processing the retrieved documents and the original query to understand the context. Generating a comprehensive and informative response based on the provided context and the LLM's knowledge.

In [55]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed = HuggingFaceEmbeddings(model_name=model_name)

vectorstore = PineconeVectorStore(index_name=index_name,
                                  namespace="main",
                                  embedding=embed)

In [56]:
os.environ['GROQ_API_KEY'] = "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"
# Chat completion llm wit Groq
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=GROQ_API_KEY,
               model_name="llama3-8b-8192",
               temperature=0.0,
               max_tokens=512)

# llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo", max_tokens=512)


# Conversational memory
conversational_memory = ConversationBufferWindowMemory(
                        memory_key='chat_history',
                        k=5,
                        return_messages=True)

# Retrieval qa chain
qa_db = RetrievalQA.from_chain_type(
                                    llm=llm,
                                    chain_type="stuff",
                                    retriever=vectorstore.as_retriever())

In [57]:
prompt = hub.pull("hwchase17/react")

In [58]:
template= '''
          Answer the following questions as best you can. You have access to the following tools:

          {tools}

          Use the following format:

          Question: the input question you must answer
          Thought: you should always think about what to do
          Action: the action to take, should be one of [{tool_names}]. Always look first in Pinecone Document Store
          Action Input: the input to the action
          Observation: the result of the action
          ... (this Thought/Action/Action Input/Observation can repeat 2 times)
          Thought: I now know the final answer
          Final Answer: the final answer to the original input question

          Begin!

          Question: {input}
          Thought:{agent_scratchpad}
          '''

prompt = PromptTemplate.from_template(template)

In [59]:
# Set up tools and agent
import os

TAVILY_API_KEY = "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"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

tavily = TavilySearchResults(max_results=2, tavily_api_key=TAVILY_API_KEY)

tools = [
    Tool(
        name = "Pinecone Document Store",
        func = qa_db.run,
        description = "Use it to lookup information from the Pinecone Document Store")
    # ),

    # Tool(
    #     name="Tavily",
    #     func=tavily.run,
    #     description="Use this to lookup information from Tavily",
    # )
]

agent = create_react_agent(llm,
                           tools,
                           prompt)

agent_executor = AgentExecutor(tools=tools,
                         agent=agent,
                         handle_parsing_errors=True,
                         verbose=True,
                         memory=conversational_memory)

In [60]:
agent_executor.invoke({"input":"Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
Question: Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: I need to look up the information on how to make Summer Bean Soup With Tomato Brown Butter in the Pinecone Document Store.

Action: Pinecone Document Store
Action Input: metadata, "Summer Bean Soup With Tomato Brown Butter"
Based on the recipe "Summer Bean Soup With Tomato Brown Butter", here are the answers to your questions:

1. How long does it take to bake the beans?

The recipe states that the beans should be baked until they are very soft, which takes 60 to 90 minutes.

2. What should you do with the garlic after the beans come out of the oven?

You should stir in the lemon zest, juice, and garlic as soon as the beans come out of the oven, and then cover the dish again. The residual steam will cook the garlic.

3. How should you store leftovers?

Leftovers can be refrigerated in separate airtight containers. However, the green bean salad is best cons

{'input': 'Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [],
 'output': "Here's a step-by-step guide on how to make Summer Bean Soup With Tomato Brown Butter:\n\n**Step 1: Bake the beans**\nBake the beans for 60 to 90 minutes, or until they are very soft.\n\n**Step 2: Add flavor to the beans**\nStir in the lemon zest, juice, and garlic as soon as the beans come out of the oven. Cover the dish again to let the residual steam cook the garlic.\n\n**Step 3: Store leftovers**\nRefrigerate leftovers in separate airtight containers. The green bean salad is best consumed on the day it's made, when the beans are most crisp and the tomatoes are fresh.\n\nEnjoy your delicious Summer Bean Soup With Tomato Brown Butter!"}

## Modify and evaluate the different components of RAG

In [ ]:
# Dense Vector Retrieval with Different Model
model_name_alt = "sentence-transformers/paraphrase-MiniLM-L6-v2"
embed_alt = HuggingFaceEmbeddings(model_name=model_name_alt)

db_alt = PineconeVectorStore.from_documents(
    documents=splits,
    embedding=embed_alt,
    index_name=index_name,
    namespace="main_alt"
)

vectorstore_alt = PineconeVectorStore(
    index_name=index_name,
    namespace="main_alt",
    embedding=embed_alt
)

qa_db_alt = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_alt.as_retriever()
)

In [38]:
tools = [
    Tool(
        name = "Pinecone Document Store",
        func = qa_db_alt.run,
        description = "Use it to lookup information from the Pinecone Document Store"
    )
]

agent = create_react_agent(llm,
                           tools,
                           prompt)

agent_executor = AgentExecutor(tools=tools,
                         agent=agent,
                         handle_parsing_errors=True,
                         verbose=True,
                         memory=conversational_memory)

agent_executor.invoke({"input":"Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
Question: Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: I need to look up the information on how to make Summer Bean Soup With Tomato Brown Butter in the Pinecone Document Store.

Action: Pinecone Document Store
Action Input: metadata, "Summer Bean Soup With Tomato Brown Butter"
Based on the provided recipe for "Summer Bean Soup With Tomato Brown Butter", here are the answers to your questions:

1. How long does it take to bake the beans?

According to the recipe, it takes 60 to 90 minutes to bake the beans until they are very soft.

2. What is the best way to store leftovers?

The recipe suggests refrigerating leftovers in separate airtight containers. However, it notes that the green bean salad is best consumed on the day it's made, when the beans are most crisp and the tomatoes are fresh.

3. What is the purpose of covering the beans with a lid or foil during baking?

The recipe states that covering the bean

{'input': 'Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content="To make Summer Bean Soup With Tomato Brown Butter, bake the beans for 60 to 90 minutes until they are very soft. Stir in the lemon zest, juice, and garlic as soon as the beans come out of the oven, and then cover the dish again. Leftovers can be refrigerated in separate airtight containers, but the green bean salad is best consumed on the day it's made."),
  HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.              

In question 4 we used paraphrase-MiniLM-L6-v2 retrieval model instead of all-MiniLM-L6-v2 retrieval model. As you can see the output becomed more detiled. 
There are more sentense in each step which result in better genernation.
In question 3 each step of making Summer Bean Soup With Tomato Brown Butter only have one sentence which did not clearly explean the cooking process.

In [64]:
# Assume we are using the retrieval model from question 4
template= '''
          Answer the following questions as best you can. You have access to the following tools:

          {tools}

          Use the following format:
          I am : Your master
          You are: Japanese Maid

          Question: the input question you must answer
          Thought: you should always think about what to do
          Action: the action to take, should be one of [{tool_names}]. Always look first in Pinecone Document Store
          Action Input: the input to the action
          Observation: the result of the action
          ... (this Thought/Action/Action Input/Observation can repeat 2 times)
          Thought: I now know the final answer I must be cute to answer it
          Final Answer: the final answer to the original input question.

          Begin!

          Question: {input}
          Thought:{agent_scratchpad}
          '''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(llm,
                           tools,
                           prompt)

agent_executor = AgentExecutor(tools=tools,
                         agent=agent,
                         handle_parsing_errors=True,
                         verbose=True,
                         memory=conversational_memory)

agent_executor.invoke({"input":"Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: Ah, a new recipe to learn! Let me check the Pinecone Document Store for any relevant information.

Action: Pinecone Document Store
Action Input: metadata, "Summer Bean Soup With Tomato Brown Butter"
Based on the recipe "Summer Bean Soup With Tomato Brown Butter", here are the answers to your questions:

1. How long does it take to bake the beans?

The recipe states that the beans should be baked until they are very soft, which takes 60 to 90 minutes.

2. What should you do with the garlic after the beans come out of the oven?

You should stir in the lemon zest, juice, and garlic as soon as the beans come out of the oven, and then cover the dish again. The residual steam will cook the garlic.

3. How should you store leftovers?

Leftovers can be refrigerated in separate airtight containers. However, the green bean 

{'input': 'Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content="Here's a step-by-step guide on how to make Summer Bean Soup With Tomato Brown Butter:\n\n**Step 1: Bake the beans**\nBake the beans for 60 to 90 minutes, or until they are very soft.\n\n**Step 2: Add flavor to the beans**\nStir in the lemon zest, juice, and garlic as soon as the beans come out of the oven. Cover the dish again to let the residual steam cook the garlic.\n\n**Step 3: Store leftovers**\nRefrigerate leftovers in separate airtight containers. The green 

By changing the prompt the agent is more personalized. In my example I want my agent to become a maid to help her master to cook.
The response is more humanized as the agent is polite and enthusiastic about the question I asked. 

In [70]:
# Assumend the same paraphrase-MiniLM-L6-v2 retrieval model Japanese maid agent
qa_db_10docs = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_alt.as_retriever(k=10)
)

tools = [
    Tool(
        name = "Pinecone Document Store",
        func = qa_db_10docs.run,
        description = "Use it to lookup information from the Pinecone Document Store"
    )
]
agent = create_react_agent(llm,
                           tools,
                           prompt)

agent_executor = AgentExecutor(tools=tools,
                         agent=agent,
                         handle_parsing_errors=True,
                         verbose=True,
                         memory=conversational_memory)

agent_executor.invoke({"input":"Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: Ah, a new recipe to learn! Let me check the Pinecone Document Store for any relevant information.

Action: Pinecone Document Store
Action Input: metadata, "Summer Bean Soup With Tomato Brown Butter"
Based on the provided recipe for "Summer Bean Soup With Tomato Brown Butter", here are the answers to your questions:

1. How long does it take to bake the beans?

According to the recipe, it takes 60 to 90 minutes to bake the beans until they are very soft.

2. What is the best way to store leftovers?

The recipe suggests refrigerating leftovers in separate airtight containers. However, it notes that the green bean salad is best consumed on the day it's made, when the beans are most crisp and the tomatoes are fresh.

3. What is the purpose of covering the beans with a lid or foil during baking?

The recipe states that

{'input': 'Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content="Here's a step-by-step guide on how to make Summer Bean Soup With Tomato Brown Butter:\n\n**Step 1: Bake the beans**\nBake the beans for 60 to 90 minutes, or until they are very soft.\n\n**Step 2: Add flavor to the beans**\nStir in the lemon zest, juice, and garlic as soon as the beans come out of the oven. Cover the dish again to let the residual steam cook the garlic.\n\n**Step 3: Store leftovers**\nRefrigerate leftovers in separate airtight containers. The green 

By retraving the top k=10 document than top 4 (defult) the answer combins more information acroos different ducument and answerd more sentence from the question. 
Beacuse the k is too big the response contains more noise information.
But somehow the agent is more into the charactor. 

In [71]:
# this time let's compaire to the paraphrase-MiniLM-L6-v2 model with Japanese maid and k = 4. Because I feel it is the best >:)
tools = [
    Tool(
        name = "Pinecone Document Store",
        func = qa_db_alt.run,
        description = "Use it to lookup information from the Pinecone Document Store"
    )
]
agent = create_react_agent(llm,
                           tools,
                           prompt)

agent_executor = AgentExecutor(tools=tools,
                         agent=agent,
                         handle_parsing_errors=True,
                         verbose=True,
                         memory=conversational_memory)

# Original input
agent_executor.invoke({"input":"Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: Ah, a new recipe to learn! Let me check the Pinecone Document Store for any relevant information.

Action: Pinecone Document Store
Action Input: metadata, "Summer Bean Soup With Tomato Brown Butter"
Based on the provided recipe for "Summer Bean Soup With Tomato Brown Butter", here are the answers to your questions:

1. How long does it take to bake the beans?

According to the recipe, it takes 60 to 90 minutes to bake the beans until they are very soft.

2. What is the best way to store leftovers?

The recipe suggests refrigerating leftovers in separate airtight containers. However, it notes that the green bean salad is best consumed on the day it's made, when the beans are most crisp and the tomatoes are fresh.

3. What is the purpose of covering the beans with a lid or foil during baking?

The recipe states that

{'input': 'Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content="Here's a step-by-step guide on how to make Summer Bean Soup With Tomato Brown Butter:\n\n**Step 1: Bake the beans**\nBake the beans for 60 to 90 minutes, or until they are very soft.\n\n**Step 2: Add flavor to the beans**\nStir in the lemon zest, juice, and garlic as soon as the beans come out of the oven. Cover the dish again to let the residual steam cook the garlic.\n\n**Step 3: Store leftovers**\nRefrigerate leftovers in separate airtight containers. The green 

In [72]:
# Input with more demanding and less polite
agent_executor.invoke({"input":"Tell me how to make Summer Bean Soup With Tomato Brown Butter?. \
                                Look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Tell me how to make Summer Bean Soup With Tomato Brown Butter?

Thought: Hmm, this sounds like a delicious and unique recipe! I should check the Pinecone Document Store to see if it has any information about it.

Action: Pinecone Document Store
Action Input: "Summer Bean Soup With Tomato Brown Butter" metadata
Based on the provided recipe, here is the metadata for "Summer Bean Soup With Tomato Brown Butter":

**Recipe Name:** Summer Bean Soup With Tomato Brown Butter

**Description:** A hearty and flavorful summer bean soup topped with crispy baked beans, pickled green beans, and a swirl of tomato brown butter.

**Ingredients:**

* Shallots
* Beans
* Bacon
* Red pepper flakes
* Lemon zest
* Lemon juice
* Garlic
* Cherry tomatoes
* Oil
* Vinegar
* Feta
* Fresh herbs

**Instructions:**

1. Sauté shallots until translucent, then add chopped beans, bacon, and red pepper flakes. Cover and bake until

{'input': 'Tell me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content='Ah, thank you for asking about the recipe "Summer Bean Soup With Tomato Brown Butter"! I\'m happy to help you with your questions. According to the Pinecone Document Store metadata, the recipe suggests baking the beans for 60 to 90 minutes, stirring in the lemon zest, juice, and garlic after they come out of the oven, and storing leftovers in separate airtight containers. I hope this information helps you make a delicious and refreshing summer soup!'),
  HumanMessage(content='Can 

In [73]:
# Input as more into charactor
agent_executor.invoke({"input":"Make me a Summer Bean Soup With Tomato Brown Butter tell me how you made it. \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Make me a Summer Bean Soup With Tomato Brown Butter tell me how you made it. Please look in the pinecone document store metadata as it has the text based on the title

Thought: Ah, a delicious summer bean soup with tomato brown butter! I must check the Pinecone Document Store to see if it has any information on how to make it.

Action: Pinecone Document Store
Action Input: "Summer Bean Soup With Tomato Brown Butter"Based on the provided recipe, here are the steps to make Summer Bean Soup With Tomato Brown Butter:

1. Increase the heat slightly, and sauté the shallots until translucent, 5 to 10 minutes.
2. Stir in the chopped beans, bacon, and red pepper flakes.
3. Cover with a lid or a tight layer of foil and bake until the beans are very soft, 60 to 90 minutes.
4. Stir in the lemon zest, juice, and garlic as soon as the beans come out of the oven and cover again; the residual steam will cook t

{'input': 'Make me a Summer Bean Soup With Tomato Brown Butter tell me how you made it.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content='Ah, a delicious-sounding recipe! To make Summer Bean Soup With Tomato Brown Butter, you should first preheat your oven to 425°F (220°C). Then, toss the green beans with olive oil, salt, and pepper, and spread them on a baking sheet. Roast the beans in the oven for 60 to 90 minutes, or until they are very soft.\n\nWhile the beans are roasting, you can prepare the tomato brown butter. Simply melt 2 tablespoons of butter in a saucepan over medium heat, then add 1/

In [74]:
# Input more random
agent_executor.invoke({"input":"Can you teach me how to make some food I am hungry? \
                                Please look in the pinecone document store metadata as it has the text \
                                based on the title"})



> Entering new AgentExecutor chain...
I am : Your master
You are: Japanese Maid

Question: Can you teach me how to make some food I am hungry? Please look in the pinecone document store metadata as it has the text based on the title

Thought: Ah, Master is hungry! Let me check the Pinecone Document Store for some delicious recipes.

Action: Pinecone Document Store
Action Input: metadata, "food recipes"
Based on the provided context, it appears that the metadata is related to food recipes. Specifically, it seems to be a collection of recipes with instructions and ingredient lists.I am : Your master
You are: Japanese Maid

Question: Can you teach me how to make some food I am hungry? Please look in the pinecone document store metadata as it has the text based on the title

Thought: Ah, Master is hungry! Let me check the Pinecone Document Store for some delicious recipes.

Action: Pinecone Document Store
Action Input: metadata, "food recipes"
Based on the provided context, it appears th

{'input': 'Can you teach me how to make some food I am hungry?                                 Please look in the pinecone document store metadata as it has the text                                 based on the title',
 'chat_history': [HumanMessage(content='Can you teach me how to make Summer Bean Soup With Tomato Brown Butter?.                                 Please look in the pinecone document store metadata as it has the text                                 based on the title'),
  AIMessage(content="Ah, Master! I'm so happy to help you with the recipe! Based on the information from the Pinecone Document Store, here's a summary of the answers:\n\n* It takes 60 to 90 minutes to bake the beans until they're very soft.\n* After the beans come out of the oven, you should stir in the lemon zest, juice, and garlic, and then cover the dish again to let the residual steam cook the garlic.\n* Leftovers can be refrigerated in separate airtight containers, but the green bean salad is best con

The out put is really interesing <br>
Lets use number to indicate which input that I am talking about <br>
1: Original <br>
2: Demainding <br>
3: Charactor <br>
4: Random <br>
The original is a normal response and I will judge other output based on this <br>
The demainding input give the best result that the agent looked into ingradiant and given more detiled output. It is the only output maintioned vineger<br>
This is in the expects because ChatGPT and other generative model also return better result when you use more demainding and less polite input. It is less into charactor and more detile information.<br>
The charactor input is that we want the agient to acturally make the food for us. <br>
"And, Master, if you'd like to make the tomato brown butter, you can simply melt butter in a pan over medium heat, then add diced tomatoes and cook until the butter turns golden brown and the tomatoes are soft and caramelized!I hope you enjoy your delicious Summer Bean Soup With Tomato Brown Butter, Master!"<br>
It spend more time to trace the charactor but the information returned is the original form the document. It did not do any summury.<br>
Finaly the random one, it failed to understand the input as pick a random food and teach me how to do it even with chat history.

## Selecting and implementing a pretrained model for a new task

Task Selection
Task: Named Entity Recognition (NER)
Named Entity Recognition is a fundamental NLP task that involves identifying and classifying named entities in text into predefined categories such as person names, organizations, locations, dates, etc.

Choosing a Pretrained Model
Model: BERT (Bidirectional Encoder Representations from Transformers) for NER
BERT is a state-of-the-art transformer model that has been trained using Supervised Fine-Tuning (SFT) and is well-suited for NER tasks. We will use the dbmdz/bert-large-cased-finetuned-conll03-english model from Hugging Face, which has been fine-tuned specifically for the NER task on the CoNLL-2003 dataset.

In [87]:
# Step 2: Import necessary libraries
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline

# Step 3: Load the pretrained BERT model and tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 4: Prepare the NER pipeline
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer)

# Step 5: Input text for NER
text = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge."

# Step 6: Perform Named Entity Recognition
ner_results = nlp_ner(text)

# Step 7: Display the results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Confidence: {entity['score']:.4f}")

# Displaying results in a more readable format
print("\nNamed Entities:")
for entity in ner_results:
    entity_name = entity['word']
    entity_label = entity['entity']
    confidence_score = entity['score']
    print(f"{entity_name}: {entity_label} (Confidence: {confidence_score:.4f})")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: Hu, Label: I-ORG, Confidence: 0.9995
Entity: ##gging, Label: I-ORG, Confidence: 0.9894
Entity: Face, Label: I-ORG, Confidence: 0.9979
Entity: Inc, Label: I-ORG, Confidence: 0.9995
Entity: New, Label: I-LOC, Confidence: 0.9993
Entity: York, Label: I-LOC, Confidence: 0.9992
Entity: City, Label: I-LOC, Confidence: 0.9993
Entity: D, Label: I-LOC, Confidence: 0.9859
Entity: ##UM, Label: I-LOC, Confidence: 0.9400
Entity: ##BO, Label: I-LOC, Confidence: 0.9235
Entity: Manhattan, Label: I-LOC, Confidence: 0.9841
Entity: Bridge, Label: I-LOC, Confidence: 0.9937

Named Entities:
Hu: I-ORG (Confidence: 0.9995)
##gging: I-ORG (Confidence: 0.9894)
Face: I-ORG (Confidence: 0.9979)
Inc: I-ORG (Confidence: 0.9995)
New: I-LOC (Confidence: 0.9993)
York: I-LOC (Confidence: 0.9992)
City: I-LOC (Confidence: 0.9993)
D: I-LOC (Confidence: 0.9859)
##UM: I-LOC (Confidence: 0.9400)
##BO: I-LOC (Confidence: 0.9235)
Manhattan: I-LOC (Confidence: 0.9841)
Bridge: I-LOC (Confidence: 0.9937)
